Question 1. (Snakes and Ladders)
The state space will be the gridded squares excluding the heads of snakes and ladders. The structure of transition probabilities is that every time you reach a new grid, you will have uniform probabilities to transit to another 6 girds (except for special grids such as the ones close to the end).

Question 2.
For example, for a game board like this:

In [2]:
# import image module
from IPython.display import Image

# get the image
Image(url="board.jpg", width=300, height=300)

Then we create the transition map for this game.

In [ ]:
from typing import Mapping, Iterable, Iterator,Sequence
from rl.markov_process import S, FiniteMarkovProcess
from rl.distribution import FiniteDistribution, Categorical, Constant
import itertools
import rl.iterate as iterate

transition_map: Mapping[S, FiniteDistribution[S]] = {
    1: Categorical({2: 1/3, 3: 1/6, 7: 1/3, 6: 1/6}),
    2: Categorical({3: 1/6, 2: 1/6, 7: 1/3, 6: 1/3}),
    3: Categorical({2: 1/6, 7: 1/3, 6: 1/3, 9: 1/6}),
    6: Categorical({7: 1/6, 6: 1/6, 9: 2/3}),
    7: Categorical({6: 1/6, 9: 5/6})
}

snakes_and_ladders: FiniteMarkovProcess = FiniteMarkovProcess(transition_map)
ini_dist: FiniteDistribution = Constant(snakes_and_ladders.non_terminal_states[0])
finish_time: Sequence[int] = []
num_play: int = 10000

def count_iterable(i: Iterable):
    return sum(1 for e in i)

def episodes_length(traces: Iterable[Iterable[S]]) -> Iterable[int]:
    for trace in traces:
        yield count_iterable(trace)

traces: Iterable[Iterable[S]] = snakes_and_ladders.traces(ini_dist)
finish: Iterator[int] = itertools.islice(episodes_length(traces), num_play)
finish_time = list(finish)

import numpy as np
import matplotlib.pyplot as plt

n, bins, patches = plt.hist(x = finish_time,
                            bins = range(3,12),
                            density=True,
                            facecolor='g',
                            alpha=0.5)
plt.grid(axis='y')
plt.xlabel('Finish time')
plt.ylabel('Frequency')
plt.title('Distribution: finish time of the game')

Question 3.
Set $E_{10} = 0$, which is the trivial step you need to jump from $10$ to $10$. Then actually we have
\begin{align*}
    E_{i} = \frac{1}{10 - i}\sum_{j = i + 1}^{10} E_{j} + 1
\end{align*}

In [59]:
F = [0] #F_{j} = E_{10 - j}
for i in range(1, 11):
    F += [1 + sum(F[:i])/i]
print(F)

[0, 1.0, 1.5, 1.8333333333333335, 2.0833333333333335, 2.2833333333333337, 2.45, 2.5928571428571434, 2.7178571428571434, 2.828968253968254, 2.9289682539682547]


Question 4.
We can set the reward function to be one every time we roll a dice and use the value function to represent the expected finishing time of the game.

In [61]:
from collections import defaultdict
from rl.markov_process import FiniteMarkovRewardProcess
reward_map: Mapping[int, Mapping[int, int]] = {
    1: Categorical({(2,1): 1/3, (3,1): 1/6, (7,1): 1/3, (6,1): 1/6}),
    2: Categorical({(3,1): 1/6, (2,1): 1/6, (7,1): 1/3, (6,1): 1/3}),
    3: Categorical({(2,1): 1/6, (7,1): 1/3, (6,1): 1/3, (9,1): 1/6}),
    6: Categorical({(7,1): 1/6, (6,1): 1/6, (9,1): 2/3}),
    7: Categorical({(6,1): 1/6, (9,1): 5/6})
}

snakes_and_ladders_mrp: FiniteMarkovRewardProcess = FiniteMarkovRewardProcess(reward_map)
snakes_and_ladders_mrp.display_value_function(gamma=1.)

{NonTerminal(state=1): 2.963,
 NonTerminal(state=2): 2.747,
 NonTerminal(state=3): 2.354,
 NonTerminal(state=6): 1.448,
 NonTerminal(state=7): 1.241}
